# Hockey Spatial Analysis: 空間統計による真の貢献度可視化プロジェクト

本ノートブックでは、MoneyPuck.com から取得した 2024 年シーズンの実ショットデータを用い、シュートの「場所の難易度」を考慮した上でゴテンダーの真の実力（GSAx）を推定します。
特に、データが限られた「小規模データ（500件）」環境において、ベイズ階層モデルがいかにして安定した意思決定を支援するかを、3層の戦略的フィルタリングを通じて実証します。

In [1]:
# 必要なライブラリ
import os
import requests
import zipfile
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.preprocessing import SplineTransformer

# 不要な出力の制御
import warnings

from src.bayesian_iroha import COLOR_PURPLE

warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from cycler import cycler

# プロジェクト共通のカラー定義（実際の運用では /src にモジュール化を推奨）
COLOR_PURPLE = "#9B5DE5"  # 事後分布・HDI
COLOR_YELLOW = "#F9C74F"  # ROPE領域
COLOR_GREEN  = "#06D6A0"  # 改善判定
COLOR_RED    = "#EF476F"  # 悪化判定
COLOR_GRAY   = "#8D99AE"  # 等価判定・参照線

def apply_brand_style():
    """分析レポートの視覚スタイルを一括適用する"""
    # カラーサイクルの設定
    palette = [COLOR_PURPLE, COLOR_YELLOW, COLOR_GREEN, COLOR_RED, COLOR_GRAY]
    plt.rcParams['axes.prop_cycle'] = cycler(color=palette)

    # 基本デザインの設定
    sns.set_style("whitegrid")
    sns.set_palette(palette)

    print("Brand Style Applied: 視覚的アイデンティティが適用されました。")

# スタイルの適用
apply_brand_style()
print("Setup: 分析環境の準備が整いました。")

Brand Style Applied: 視覚的アイデンティティが適用されました。
Setup: 分析環境の準備が整いました。


分析に必要な統計・可視化ライブラリをロードし、レポートの品質を担保するためのデザイン設定（`Seaborn` / `Japanize-Matplotlib`）を行います。

> Tips:
>
> 分析結果のビジュアルを統一し、どのグラフを見ても「紫は実力、緑は成功」と直感的に理解できるようにしています。
>
>
> Color Cycle
> グラフを描画する際、色が指定されていない場合に自動的に適用される色の順番です。
> 最初は紫、次を黄色...と決めておくことで、毎回指定しなくても色が揃います。
>
>
> ビジネスレポートにおいて「色の意味」を固定することは、意思決定のスピードを劇的に高めます。
> 複数の Notebook を管理する実務現場では、これを共通モジュール化し、メンテナンス姓を高めることを推奨します。

## 1. データ準備（Preparation）
外部ソース（[Moneypuck.com](https://moneypuck.com/)）から ZIP形式の巨大なデータセットを自動取得し、解析可能な状態に展開します。


In [4]:
# データのダウンロード（MoneyPuck 2024 shots Data）
URL_DATA = "https://peter-tanner.com/moneypuck/downloads/shots_2024.zip"
ZIP_NAME = "shots_2024.zip"
DIR_EXTRACT = "data_raw"

print(f"Data Preparation: Downloading from {URL_DATA}...")
response = requests.get(URL_DATA)
with open(ZIP_NAME, "wb") as f:
    f.write(response.content)

# ZIP の展開
print(f"Data Preparation: Extracting {ZIP_NAME}...")
with zipfile.ZipFile(ZIP_NAME, "r") as zip_ref:
    zip_ref.extractall(DIR_EXTRACT)

# 展開ファイルの確認
files = os.listdir(DIR_EXTRACT)
print(f"Data Preparation: Extracted files: {files}")

In [5]:
# データのプレビュー（ホワイトボックス化）
CSV_TARGET = os.path.join(DIR_EXTRACT, "shots_2024.csv")

# 先頭５行を確認
df_preview = pd.read_csv(CSV_TARGET, nrows=5)
print("Data Preparation: Preview of the data:")
df_preview.head()

Data Preparation: Preview of the data:


,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,...,xCordAdjusted,xFroze,xGoal,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted
0,0,52.0,57.0,57.0,-41.0,41.0,0.0,0,0,0,...,57,0.238455,0.012537,0.394229,0.301072,0.022807,0.030900,0.710867,-40,-40
1,1,33.0,71.0,71.0,-28.0,28.0,-6.0,0,0,0,...,71,0.198306,0.021962,0.404919,0.313773,0.023774,0.037266,0.759039,-28,-28
2,2,48.0,48.0,48.0,-24.0,24.0,-12.6,0,0,0,...,48,0.213829,0.028057,0.405311,0.294682,0.025849,0.032272,0.696901,-24,-24
3,3,58.0,-40.0,40.0,-31.0,31.0,0.0,0,0,0,...,41,0.209478,0.009832,0.449775,0.277671,0.019667,0.033577,0.610530,-31,31
4,4,56.0,-35.0,35.0,15.0,15.0,0.0,0,0,0,...,36,0.376712,0.028884,0.307725,0.205568,0.022266,0.058845,0.799576,15,-15


膨大なカラムを持つデータから、空間的な「期待ゴール率」と「ゴテンダー評価」に直結する最小限の変数のみを抽出してロードします。

In [7]:
# 利用する列を展開してロード（メモリ効率化）
COLS_USE = [
    "shotID",
    "goal",  # 1: Goal, 0: No Goal
    "xCordAdjusted",  # 調整済み x 座標
    "yCordAdjusted",  # 調整済み y 座標
    "goalieIdForShot",  # ゴテンダー ID
    "goalieNameForShot"  # 選手名
]

print("Data Preparation: Loading real dataset from CSV...")
df_raw = pd.read_csv(CSV_TARGET, usecols=COLS_USE)

print(f"Data Preparation: Data load finished. {len(df_raw):,} rows")
df_raw.head()

Data Preparation: Loading real dataset from CSV...
Data Preparation: Data load finished. 119,870 rows


,shotID,goal,goalieIdForShot,goalieNameForShot,xCordAdjusted,yCordAdjusted
0,0,0,8480045,Ukko-Pekka Luukkonen,57,-40
1,1,0,8480045,Ukko-Pekka Luukkonen,71,-28
2,2,0,8480045,Ukko-Pekka Luukkonen,48,-24
3,3,0,8474593,Jacob Markstrom,41,31
4,4,0,8474593,Jacob Markstrom,36,-15


投影（Projection / usecols）
- 巨大なデータセットから、特定の属性（列）のみの選択をしてメモリに読み込むようにします。
データ全てを覚えるのではなく、必要なカラムだけを抽出することで作業効率を劇的に上げるテクニックです。

## データ加工（Processing）
キーバーがいない状況でのゴールなど、能力評価においてノイズとなるデータを除外します。

In [8]:
# クレンジング:
df_clean = df_raw.dropna(subset=["goalieIdForShot"]).copy()  # ゴテンダー不在（Empty Net）データの厳密な除外
df_clean = df_clean[df_clean["goalieIdForShot"] > 0].copy()  # MoneyPuck データでは無人ゴール時に ID が 0 や欠損になるため、これを除外します。

print(f"Data Processing: {len(df_clean):,} rows left after cleaning.")

Data Processing: 118,900 rows left after cleaning.


データが少ない環境（新規事業や特定セグメントの分析）を再現し、不可実性下での判断プロセスを構築します。

In [9]:
# 小規模データ化（３桁台のサンプリング）
# 意思決定の難易度が高い「事例が少ない」状況を再現するため、あえて 500 件に絞ります
SIZE_SAMPLE = 500
df_small = df_clean.sample(n=SIZE_SAMPLE, random_state=42).reset_index(drop=True)

df_small

,shotID,goal,goalieIdForShot,goalieNameForShot,xCordAdjusted,yCordAdjusted
0,77213,0,8480947,Kevin Lankinen,57,-17
1,62564,1,8482487,Jakub Dobes,42,12
2,50703,0,8480382,Alexandar Georgiev,76,10
3,80853,1,8479406,Filip Gustavsson,72,14
4,65495,0,8481020,Justus Annunen,23,-39
...,...,...,...,...,...,...
495,43089,0,8475683,Sergei Bobrovsky,56,-12
496,50168,0,8481611,Pyotr Kochetkov,66,35
497,15962,0,8480280,Jeremy Swayman,62,16
498,68658,0,8475683,Sergei Bobrovsky,37,11


カテゴリ変数（ID）を確率モデルが扱える形式に変換し、結果の解釈をしやすくするための紐付けを行います。

In [14]:
# インデックス化
# 選手ID を 0 からの連番に変換し、ID から名前を引く辞書を作成します
codes_goalie, uniques_goalie = pd.factorize(df_small["goalieIdForShot"])
df_small["idx_goalie"] = codes_goalie
df_small["obs_shots"] = df_small["goal"].astype(int)

# ID から名前を引くためのマップ
id_to_name = df_small.set_index("idx_goalie")["goalieNameForShot"].to_dict()

print(f"Data Processing: unique count of Goalie {len(uniques_goalie):,}")
df_small[["goalieIdForShot", "idx_goalie", "obs_shots"]]

Data Processing: unique count of Goalie 79


{0: 'Kevin Lankinen',
 1: 'Jakub Dobes',
 2: 'Alexandar Georgiev',
 3: 'Filip Gustavsson',
 4: 'Justus Annunen',
 5: 'Charlie Lindgren',
 6: 'James Reimer',
 7: 'Samuel Ersson',
 8: 'Joseph Woll',
 9: 'Stuart Skinner',
 10: 'Philipp Grubauer',
 11: 'Lukas Dostal',
 12: 'Connor Ingram',
 13: 'Darcy Kuemper',
 14: 'Anton Forsberg',
 15: 'Andrei Vasilevskiy',
 16: 'Frederik Andersen',
 17: 'Cam Talbot',
 18: 'Jeremy Swayman',
 19: 'Sam Montembeault',
 20: 'Jordan Binnington',
 21: 'Juuse Saros',
 22: 'Jacob Markstrom',
 23: 'Ukko-Pekka Luukkonen',
 24: 'Tristan Jarry',
 25: 'Sergei Bobrovsky',
 26: 'Mackenzie Blackwood',
 27: 'Alex Lyon',
 28: 'Igor Shesterkin',
 29: 'Dustin Wolf',
 30: 'Pyotr Kochetkov',
 31: 'Jonathan Quick',
 32: 'Connor Hellebuyck',
 33: 'Jake Oettinger',
 34: 'Ilya Samsonov',
 35: 'Eric Comrie',
 36: 'Spencer Martin',
 37: 'Elvis Merzlikins',
 38: 'Yaroslav Askarov',
 39: 'David Rittich',
 40: 'Alex Nedeljkovic',
 41: 'Leevi Merilainen',
 42: 'Dennis Hildeby',
 43: '

座標（$x$, $y$）とう数値を、モデルが「どのエリアが危険か」を滑らかに学習できる高度な特徴量に変換します。

In [15]:
df_small.columns

Index(['shotID', 'goal', 'goalieIdForShot', 'goalieNameForShot',
       'xCordAdjusted', 'yCordAdjusted', 'idx_goalie', 'obs_shots'],
      dtype='object')

In [16]:
# 特徴量生成（空間基底関数の適用）
# 座標データから滑らかな空間曲面（スプライン基底）を生成します
spline = SplineTransformer(n_knots=5, degree=3, include_bias=False)
basis_spatial = spline.fit_transform(df_small[["xCordAdjusted", "yCordAdjusted"]])

print(f"Data Processing: 特徴量行列の計上 {basis_spatial.shape}")

Data Processing: 特徴量行列の計上 (500, 12)


このセルでは、「シュート位置（x, y）」という単純な 2 つの数値を、この SplineTransformer に通すことで、
**「ゴール付近ならこの値が大きくなる」「端の方ならこの値が変化する」といった、
空間的な特徴を持つ複数の変数（多次元の行列）**へと変換しています。


これにより、モデルは「座標 (x, y)」をただの数字としてではなく、
「リンク上のどのエリアがゴールになりやすいか」という滑らかな非線形の関係として学習できるようになります。

### スプライン基底関数
一言でいうと、**「複雑な1つの曲面を、いくつかの『シンプルな山の形（部品）』の組み合わせで表現する仕組み」**
 のことです。

#### 1.  「基底（きてい）」とは「部品」のこと
例えば、絵の具で「オレンジ色」を作るとき、「赤」と「黄色」を混ぜます。このとき、赤と黄色が**「基底（部品）」**です。
データ分析における基底関数も同じです。 「座標 (x, y)」という生データをそのまま使うのではなく、**「特定のエリアに反応するいくつかの部品（関数）」**に変換します。

#### 2. スプライン基底関数の仕組み（イメージ）
アイスホッケーのリンクを想像してください。
1. エリアの分割: リンクの上に、いくつかの「小さな山（テントのような形）」を等間隔に並べます。
2. 反応する場所:
    - ゴール正面にある「山A」は、シュートが正面から打たれたときだけ高い値を出します。
    - サイドにある「山B」は、サイドから打たれたときだけ反応します。


合成: これらの「山（部品）」をそれぞれ「どれくらい重視するか（重み）」を掛け合わせて足し算すると、リンク全体の「シュートの危険度マップ」が完成します。
この**「一つ一つの山」が「スプライン基底関数」**です。

#### 3. なぜ「スプライン」なのか？
単にエリアを四角く区切る（＝モザイク状にする）だけだと、エリアの境界線で値が突然跳ね上がってしまい、不自然です。


**「スプライン」** という手法を使うと、隣り合う「山」同士が滑らかに重なり合うように設計されます。これにより：

- 「ここから急にゴールしやすくなる」という不自然な段差がなくなる。
- **「ゴールに近づくにつれて、なだらかに危険度が高まる」** という自然な表現が可能になる。

#### 4. SplineTransformer がやっていること
コードにある以下の処理は、まさにこの「部品への変換」を行っています。
```
# 座標 (x, y) を、複数の「山の反応度」に変換する
basis_spatial = spline.fit_transform(df_small[["xCordAdjusted", "yCordAdjusted"]])
```

- 入力: [x座標, y座標] （2つの数字）
- 出力: [山Aの反応, 山Bの反応, 山Cの反応, ...] （多くの数字の列）

の変換のおかげで、あとの統計モデル（PyMC）は「座標から直接計算する」という難しいことをしなくて済み、
**「どの山の重みを大きくすれば、実際のゴール率と一致するか？」** を計算するだけで済むようになります。

#### まとめ
**スプライン基底関数**とは、
複雑な空間の形を捉えるために、**「滑らかに重なり合った、場所ごとの反応パーツ」**に分解して表現する手法のことです。

`SplineTransformar(n_knots=5, degree=3, include_bias=False)`

sciket-learn で提供されている、数値データ。今回は、座標から「スプライン基底関数」と呼ばれる特徴量を生成。

Args:
> `n_konots=5` (ノットの数)
> ノット（データを分割する「結び目（ノット」））の数。
> スプライン曲線は、データをいくつかの区間に分けて、それぞれの区間で多項式を当てはめます。
> この値を大きくするほど、より細かく複雑な形状（曲面）を表現できるようになりますが、
> 大きくしすぎると過学習（ノイズに反応しすぎること）のリスクが高まります。

>degree=3 (多項式の次数)
>各区間で使用する多項式の次数。
>- degree=1: 線形（カクカクした折れ線）
>- degree=2: 2次関数（放物線）
>- degree=3: 3次関数（立方スプライン）
>
>このコードでは: 一般的に最もよく使われる 3（3次スプライン） が指定されています。
これにより、結び目の境界でも変化が非常に滑らか（数学的に2回微分可能）な曲面を作ることができます。

>include_b=False (バイアス項を含めるか)
全ての基底の和が 1 になるような定数項（インターセプト）を特徴量に含めるかどうか。
True にすると、生成される行列に「常に 1 となる列」のような定数成分が含まれます。
このコードでは: False に設定されています。
これは、後続の統計モデル（PyMCなど）側で別途切片（Intercept）を用意する場合や、
冗長な変数を避けるためによく取られる設定です。